## **Invisibility Cloak**
Project idea: https://harshilp.medium.com/invisibility-cloak-using-opencv-8b07142c83d6

This simple project aims to imitate the effects of invisibility cloak (like in Harry Potter). The technique that will be used is opposite to the Green Screening. In green screening, we remove background but here we will remove the foreground frame.

1. Capture and store the background frame
2. Detect the defined color using color detection and segmentation algorithm.
3. Segment out the defined colored part by generating a mask.
4. Generate the final augmented output to create a magical effect. [ output as .avi file ]

In [5]:
# import necessary libraries
import cv2
import time
import numpy as np

import matplotlib.pyplot as plt

The HSV value of the color will be updated accordingly.
### **Hue Saturation Value**
### **H : Hue**
Hue is the colour portion of the model, expressed as a number from 0 to 360 degrees:

- Red falls between 0 and 60 degrees.
- Yellow falls between 61 and 120 degrees.
- Green falls between 121–180 degrees.
- Cyan falls between 181–240 degrees.
- Blue falls between 241–300 degrees.
- Magenta falls between 301–360 degrees.

### **S : Saturation**
Saturation describes the amount of grey in a particular colour, from 0 to 100 percent. Reducing this component toward zero introduces more grey and produces a faded effect. Sometimes, saturation appears as a range from just 0–1, where 0 is grey, and 1 is a primary colour.

### **V : Value (Brightness)**
Value works in conjunction with saturation and describes the brightness or intensity of the colour, from 0–100 percent, where 0 is completely black, and 100 is the brightest and reveals the most colour.


### **Steps**
1. Recording and caching the background for each frame
2. Detecting the define color portion in each frame
3. Replacing the red portion with a mask image in each frame

In [9]:
## 1.
# Reading video feed from webcam
cap = cv2.VideoCapture(0) # default channel

# Automatically grab width and height from video feed
# (returns float which we need to convert to integer for later on!)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Preparation for writing output video
fourcc = cv2.VideoWriter_fourcc(*'DIVX') # 'DIVX' for Windows
writer = cv2.VideoWriter('invisibility_cloak.avi',fourcc,20,(width,height))

# Allow system to sleep for 3 seconds before the webcam starts
time.sleep(3)
count = 0
background = 0

# Capture the bacground in range of 60
for i in range(60):
    ret, background = cap.read()
background = np.flip(background, axis=1)

## 2.

# Read every frame from the webcam, until the camera is open
while(cap.isOpened()):
    ret, frame = cap.read()
    if not ret: # no frame retrieved
        break

    count += 1
    frame = np.flip(frame, axis=1)

    # Convert color space from BGR (OpenCV default) to HSV
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Generate masks to detect specific color (in this case red)
    lower_bound = np.array([0, 125, 50])
    upper_bound = np.array([10, 255, 255])
    mask1 = cv2.inRange(hsv, lower_bound, upper_bound) # Checks for parts of hsv img that lie between range of lower and upper red.

    lower_bound = np.array([170, 120, 70])
    upper_bound = np.array([180, 255, 255])
    mask2 = cv2.inRange(hsv, lower_bound, upper_bound)

    mask1 = mask1 + mask2

## 3.

    # Open and Dilate the mask image
    mask_morph = cv2.morphologyEx(mask1, cv2.MORPH_OPEN, np.ones((3,3), np.uint8)) # removes noise (erosion => dilation)
    mask_morph = cv2.morphologyEx(mask1, cv2.MORPH_DILATE, np.ones((3,3), np.uint8)) # expand mask

    # Create an inverted mask to segment out red color from the frame
    mask_inv = cv2.bitwise_not(mask_morph)

    # Segment the red color part out of the grame using bitwise_and with the inverted mask
    res1 = cv2.bitwise_and(frame, frame, mask=mask_inv)

    # Create image showing static background grame pixels only for masked region
    res2 = cv2.bitwise_and(background, background, mask=mask_morph)

    # Generate final output and write
    final_output = cv2.addWeighted(src1=res1, alpha=1, src2=res2, beta=1, gamma=0) # blend 2 image
    writer.write(final_output)
    cv2.imshow("invisibility_cloak", final_output)

    # Quit with the "q" button on a keyboard.
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
writer.release()
cv2.destroyAllWindows()

In [15]:
# plt.imshow(mask1, cmap='gray')

In [14]:
# plt.imshow(mask2, cmap='gray')